In [1]:
from bs4 import BeautifulSoup
import requests

# Extract the content of the HTML page and create soup object for scrapping

In [2]:
text = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(text, "lxml")

# Create object table from HTML tag "table" with class attribute "wikitable sortable"

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})


# Extract all the table rows from the HTML table and remove the headers row

In [4]:
rows = table.find_all('tr')
rows.pop(0)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

# Logic to create the dataframe with the specified requirements

In [5]:
import pandas as pd
l = []
i=0
    
for tr in rows:
    td = tr.find_all('td')
    
    if td[1].text != 'Not assigned':
        
        if td[2].text == 'Not assigned\n':
            td[2] = td[1]
            
        row = [tr.text for tr in td]
        row[2] = row[2].rstrip()
        
        if i == 0:
            l.append(row)
            i = i+1
            continue

            
        else:
            if l[i-1][1] == row[1]:
                l[i-1][2] =  l[i-1][2] +","+row[2]
                continue

        i = i+1
        l.append(row)
        
df = pd.DataFrame(l, columns=["Postal Code", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,"Parkwoods,Victoria Village"
1,M5A,Downtown Toronto,"Harbourfront,Regent Park"
2,M6A,North York,"Lawrence Heights,Lawrence Manor"
3,M7A,Queen's Park,Queen's Park
4,M9A,Etobicoke,Islington Avenue


# Shape of the dataframe

In [6]:
df.shape

(85, 3)

# Part II

# Download csv file with coordinates and create a second dataframe

In [17]:
df2 = pd.read_csv("http://cocl.us/Geospatial_data")
df2 = pd.merge(df, df2, on='Postal Cose')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
